### Imports

In [1]:
from UNet import *
from utils_V3 import *

ds = xr.open_dataset('data_850/2022_850_SA_coarsen.nc')
ds.load()

c:\Users\23603526\AppData\Local\Programs\Python\Python311\Lib\site-packages\xarray\backends\plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset> Size: 197MB
Dimensions:    (time: 8760, latitude: 16, longitude: 32)
Coordinates:
  * time       (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31T23:00:00
  * longitude  (longitude) float32 128B 16.31 16.81 17.31 ... 30.81 31.31 31.81
  * latitude   (latitude) float32 64B -34.89 -34.39 -33.89 ... -27.89 -27.39
Data variables:
    q          (time, latitude, longitude) float64 36MB 0.007599 ... 0.009553
    t          (time, latitude, longitude) float64 36MB 282.4 282.5 ... 284.3
    u          (time, latitude, longitude) float64 36MB 14.09 13.71 ... 3.188
    v          (time, latitude, longitude) float64 36MB -0.8432 -1.022 ... 4.144
    w          (time, latitude, longitude) float64 36MB -0.002005 ... 0.1457
    wspd       (time, latitude, longitude) float32 18MB 14.12 13.75 ... 5.228
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-28 12:36:03 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
    wspd_units:   m/s

### Model Setup

In [8]:
model_class = WeatherMLModel(ds=ds, window_size=24, steps=12)

Windowed...
Splitting...
Shuffling...
Data prepared...
Class setup done...


In [9]:
def build_RRnet_time(input_shape, steps = 1):
    input_data = Input(input_shape)

    value = 1

    s1, p1 = RR_encoder_block(input_data, 32 * value)
    s2, p2 = RR_encoder_block(p1, 64 * value)
    s3, p3 = encoder_block(p2, 128 * value)
    s4, p4 = encoder_block(p3, 256* value)

    b1 = RR_block(p4, 512* value) #Bridge

    d1 = decoder_block(b1, s4, 256* value)
    d2 = decoder_block(d1, s3, 128* value)
    d3 = RR_decoder_block(d2, s2, 64* value)
    d4 = RR_decoder_block(d3, s1, 32* value)

    time_input = Input(shape=(input_shape[2],))

    output = Conv2D(steps, 1, padding="same", activation="sigmoid")(d4)  #Binary (can be multiclass)

    model = Model(inputs = [input_data, time_input], outputs=output, name="RR-Net")
    return model

In [12]:
model_class.assign_model(build_RRnet_time(model_class.X_train_tensor.shape[1:], model_class.y_train_tensor.shape[-1]))
model_class.load_model('models/09_12_08_40.h5')

Model assigned...


In [15]:
model_class.plot_predictions(seed=150, frame_rate=4, levels=10)

1/1 [==============================] - 0s 92ms/step
